# Agrupando e manipulando dados com Pandas

É muito comum em Data Science nós termos de responder perguntas como:
- 'Qual a média salarial **por sexo**?'
- 'Qual a média salarial **por região do Brasil**?'
- 'Qual a nota máxima do ENEM **por ano**?'


Entre muitas outras similares.

Vemos que todas essas perguntas possuem uma similaridade, que é a parte do final: 'por sexo', 'por cor', 'por região', 'por idade', 'por cargo', 'por ano'...
Em geral, queremos avaliar uma métrica de acordo com cada **categoria** ou **grupo**.

Para responder esse tipo de pergunta, precisamos **agrupar** nossos dados de acordo com cada categoria.

O *pandas* possui uma funcionalidade muito bacana para nos ajudar, que é a função *groupby*.

![img](https://github.com/Giatroo/BeeData_GroupBy-in-Pandas/blob/main/split-apply-combine.jpeg?raw=true)

Primeiro, importamos o pandas, nosso foco hoje, e o numpy, que vai nos ser útil mais para frente.

In [ ]:
import numpy as np
import pandas as pd

Antes de mais nada, as versões do numpy e pandas que utilizei para fazer esse notebook são, respectivamente, 1.19.2 e 1.2.3.

Isso é importante, pois bibliotecas costumam mudar conforme o tempo e algo que eu mostrar aqui pode ficar ultrapassado no futuro.

In [ ]:
print(f'A versão do numpy é : {np.__version__}')
print(f'A versão do pandas é : {pd.__version__}')

Vamos agora importar nossa tabela.

Vou utilizar uma tabela do censo americado de 2010 a 2015.

In [ ]:
df = pd.read_csv('../input/census/census.csv')
df.head()

São 100 colunas ao todo, então vamos remover as que não nos interessam e manter apenas os nascimentos de cada ano e estimativa da população. Além disso, vou manter apenas as linhas onde o *sumarrary level* é *50*, pois essas são as linhas que se referem ao census de cada cidade.

In [ ]:
colunas = ['STNAME', 'CTYNAME', 'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
          'BIRTHS2015', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 
          'POPESTIMATE2014', 'POPESTIMATE2015']
df = df[df['SUMLEV'] == 50]
df = df[colunas]
df.head()

In [ ]:
df.shape

# Formas de agrupar dados

Vamos agora entender como agrupar esses dados **por estado**. Suponha por exemplo, que queremos saber a média da população em cada estado, ou então a soma, ou mesmo os valores máximos e mínimos de cada cidade por estado.

Para começar a fazer essas análises, precisamos, antes de mais nada, agrupar essas 3142 linhas de acordo com os estados.

In [ ]:
df['STNAME'].unique()

Aqui vemos que a coluna *STNAME* possui todos os estados dos EUA, mas que várias linhas são referente ao Alabama, várias linhas são referentes à Florida e assim por diante. A primeira forma que veremos de agrupar é justamente colocar no mesmo grupo aquelas linhas que têm o mesmo valor para uma determinada coluna.

In [ ]:
# Utilizamos a função groupby passando o nome da coluna que queremos usar para agrupar
g = df.groupby('STNAME')

In [ ]:
# Se tentarmos mostrar a variável g, vemos que ela é um objeto do tipo pandas.core.groupby.generic.DataFrameGroupBy
# Mas ainda não conseguimos entender o que realmente aconteceu com o nosso DataFrame
g

A variável `g` é um **objeto** como vários outro do *pandas* (e.g. uma `Series` ou um `DataFrame`). Isso significa que temos várias funções e atributos dentro desse objeto que nós podemos usar.

Por exemplo, o método `describe` vai nos gerar várias informações, como em um `DataFrame` normal, mas de forma agrupada por estado.

In [ ]:
g.describe().head(10)

Veja que, para cada coluna, temos várias informações. Por exemplo, temos a média, desvio padrão, máximo, mínimo, mediana para a população de 2015:

In [ ]:
g.describe()['POPESTIMATE2015'].head(10)

**Importante:** note que, depois de aplicarmos a função `describe`, o resultado é um `DataFrame`. Ou seja, podemos usar tudo aquilo que já sabemos sobre `DataFrame`'s (como acessar uma coluna, igual eu fiz acima).

In [ ]:
type(g.describe())

O `describe` nos traz muitas informações, mas podemos também aplicar outras funções para ter apenas aquilo que desejamos (e de forma mais rápida):

In [ ]:
g.mean().head()

In [ ]:
g.max().head()
# Observe que quando uma função não pode ser aplicada a uma coluna, essa coluna não aparece no resultado final.
# Nesse caso, 'max' pode ser aplicada a CTYNAME utilizando a ordem lexográfica.

In [ ]:
g.count().head()

A lista completa de operações pode ser encontrada [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats).

# Acessando grupos

Em muitos casos, nós precisamos acessar um grupo específico ou mesmo iterar sobre nossos grupos. 

Para acessar um grupo específico, usamos a função `get_group` e passamos o grupo desejado.

In [ ]:
g.get_group('California').head(10)

Note novamente que o resultado é um `DataFrame`. Nesse caso, todos os valores da coluna *STNAME* são 'California'.

O resultado é exatamente o mesmo que quando fazemos:

In [ ]:
df[df['STNAME'] == 'California'].head(10)

A diferença é que `g` possui muitos outros grupos e é muito mais prático, como já vimos.

Para iterar pelos grupos, podemos usar um `for`.

In [ ]:
# Para iterar é como se fosse um dicionário com a chave do grupo e o DataFrame
for chave, grupo in g:
    print(f'O nome do grupo é {chave} e o shape é {grupo.shape}.')

Espero que agora esteja começando a ficar claro quais são as funcionalidade desse tipo de objeto.

Para terminar essa primeira parte, vale a pena dizer que podemos indexar colunas de um objeto `groupby`:

In [ ]:
g['BIRTHS2013'].max().head()

In [ ]:
g[['BIRTHS2010','BIRTHS2011','BIRTHS2012','POPESTIMATE2014','POPESTIMATE2015']].mean().head()

# Outras formas de se agrupar

Agora que já entendemos alguns usos básicos de um agrupamento. Vamos voltar um pouco para a função `groupby` e ver rapidamente outras formas de utiliza-la.

### Várias colunas
A primeira forma é passando um array de colunas ao invés de uma única coluna.

In [ ]:
# Vamos criar um dataframe menor para ajudar na visualização:
df1 = pd.DataFrame({'col1' : [1, 1, 2, 2, 2, 3],
                  'col2' : ['no', 'yes', 'no', 'no', 'yes', 'no'],
                  'col_val1' : np.random.rand(6),
                  'col_val2' : np.random.randn(6)*10})
df1

In [ ]:
df1.groupby(['col1', 'col2']).mean()

Perceba que agora temos um `DataFrame` com um `MultiIndex` onde o nível externo é *'col1'* e o interno é *'col2'*. E note que não há uma linha `(3, 'yes')` porque essa combinação não existe.

**Obs.:** podemos passar uma lista com quantas colunas nós quisermos, não precisa parar em duas.

### Por nível em um MultiIndex
Falando em `MultiIndex`, outra forma de agrupar é informando o nível em um `MultiIndex`.

In [ ]:
colunas = pd.MultiIndex.from_arrays([['US']*3 +['BR']*2,
                                     [1, 3, 5, 1, 3]],
                                    names=['pais', 'num'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=colunas)
hier_df

In [ ]:
# groupby aceita os keyword arguments level e axis
hier_df.groupby(level='pais', axis='columns').max()

**Obs.:** como queremos agrupar pelos países (que são 'US' e 'BR') e eles são colunas, precisamos definir `axis='columns'`. Se tivéssemos um `MultiIndex` nas linhas e quisessemos agrupar nas linhas, varíamos `axis='index'` ou simplesmente não falaríamos nada (pois, `axis='index'` é o padrão). [Aqui](https://stackoverflow.com/questions/22149584/what-does-axis-in-pandas-mean) tem umas discussão interessante sobre a diferença entre os *eixos* (axis).

### Usando uma função

Podemos também agrupar utilizando uma função (criada por nós ou não). 

A função vai receber o *índice de cada linha* e deve devolver um valor que pode ser comparável. Então vamos criar os grupos de acordo com os resultados iguais da função.

In [ ]:
df2 = pd.DataFrame({'nome' : ['João', 'Cláudia', 'Mariana', 'Mario', 'Joana', 'Lucas', 'Marcos'],
                   'nota1' : np.random.randint(0, 11, size=7),
                   'nota2' : np.random.randint(0, 11, size=7)})
df2 = df2.set_index('nome') # É importante que o índice seja os nomes, pois a função vai receber esses nomes
df2

In [ ]:
# Vamos criar uma função que recebe uma string e retorna suas primeiras duas letras
def duas_letras(s):
    # Sempre que tiver dúvida quanto a o que a função está recebendo, experimente printar o valor ou seu tipo
    #print(s)
    #print(type(s))
    return s[:2]

# Passamos nossa função para groupby 
df2.groupby(duas_letras).max()

Veja como nós agrupamos as pessoas de acordo com as duas primeiras letras de seu nome.

Poderíamos usar também funções do Python, como a função `len`:

In [ ]:
df2.groupby(len).max()

Se quisermos ver quem percente a cada grupo, uma forma prática é utilizando o *atributo* `groups` do nosso objeto `groupby`.

In [ ]:
df2.groupby(len).groups

# Agregações

Vamos agora passar para as principais aplicações e utilizades dos agrupamentos:
- Gerar agregações
- Gerar transformações
- Filtrar grupos

Começando com as agregações.

Sempre que temos uma lista e geramos um número com base naquela lista, estamos gerando uma agregação. Ou seja, uma agregação é sempre uma operação que transforma uma lista de valores em um único valor. Exemplos de agregação são soma, média, mediana, desvio padrão, máximo, mínimo, produto, entre outras agregações que nós mesmos podemos criar (agregações não se restringem a números).

Quando se trata de grupos, as agregações vão ser feitas dentro de cada grupo. Ou seja, se utilizamos uma média, vamos ter a média dos valores de cada grupo.

No pandas, as funções `agg` ou `aggregate` são utilizadas para fazer agregações (não existe diferença nenhuma entre elas, `agg` é um *alias* para *aggregate*).

![image](https://github.com/Giatroo/BeeData_GroupBy-in-Pandas/blob/main/split-apply-combine2.jpg?raw=true)

In [ ]:
# Vamos voltar ao nosso banco de dados do censo
g.agg(np.mean).head()

Note que isso é exatamente o que já fizemos anteriormente:

In [ ]:
g.mean().head()

Mas então qual a diferença? Fazer apenas `.mean()` parece muito mais fácil.

A diferença é que a função `agg` nos permite fazer muitos mais. E é isso que vamos ver agora:

### Várias agregações
Com `agg` podemos passar quantas funções de agregação nós quisermos e vamos gerar um DataFrame com colunas hierarquicas com os nomes dessas funções:

In [ ]:
g.agg([np.mean, np.sum]).head(10)

### Agregações customizáveis
Além disso, podemos passar nossas próprias funções para a `agg`:

In [ ]:
# O exemplo é besta, mas em muitos casos nós precisamos definir nossas próprias funções para gerar agregações 
# exatamente como desejamos

def meia_media(s):
    return s.mean() / 2
def soma_mais_10(s):
    return s.sum() + 10

g.agg([meia_media, soma_mais_10]).head(10)

**Observe** que a função passada para `agg` recebe como parâmetro uma `Series` que corresponde a uma coluna do agrupamento. Vamos provar isso com `DataFrame` menor:

In [ ]:
df2

In [ ]:
def verifica_tipo(s):
    print(type(s))
    print(s)
    print('-' * 100, end='\n\n')
    if s.shape[0] > 1:
        return 1000
    else:
        return 0
    
df2.groupby(duas_letras).agg(verifica_tipo)

Veja como cada grupo aparece duas vezes, um para cada coluna do `DataFrame` original.

### Nomear agregações
Podemos, agora, querer renomear os nomes que aparecem em cada coluna da tabela agregada. Veja como podemos ter alguns casos em que renomar as colunas é bastante desejável:

In [ ]:
def uma_funcao_com_um_nome_desnecessariamente_grande(s):
    return s.mean() / 2

g.agg([uma_funcao_com_um_nome_desnecessariamente_grande,
       lambda x : x.sum(),
       lambda x : x.max() - x.min()]).head(10)

Veja que temos uma função com um nome gigante e duas funções lambda (funções sem nome) que acabam não dando um resultado muito desejável. 

Podemos, entretanto, passar nomes para nossas colunas através da função `agg`. 

Ao invés de passar uma lista de funções, passamos uma lista de tuplas onde o primeiro elemento é o nome e o segundo,a função:

In [ ]:
g.agg([('meia_soma', uma_funcao_com_um_nome_desnecessariamente_grande),
       ('soma', lambda x : x.sum()),
       ('amplitude', lambda x : x.max() - x.min())]).head(10)

Existem outras formas de nomear, mas elas são muito mais complexas e eu acho que não vale a pena aprender.

Entretanto, recomendo você dar uma [olhada](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation).

### Agregações diferentes para colunas diferentes
Para finalizar as agregações, outra coisas que a função `agg` nos permite é aplicar agregações por coluna.

Note que eu passei três funções para `agg` e elas foram aplicadas em todas as colunas numéricas do meu `DataFrame`. Muitas vezes, não queremos isso. Pode ser que eu queira a soma dos nascimentos e a média e amplitude da quantidade de população (diferentes agregações para diferentes colunas). 

Podemos fazer isso passando um dicionário para `agg`:

In [ ]:
minhas_agg = {'BIRTHS2013' : np.sum,
              'BIRTHS2014' : np.sum,
              'BIRTHS2015' : np.sum,
              'POPESTIMATE2013' : [np.mean, lambda x : x.max() - x.min()],
              'POPESTIMATE2014' : [np.mean, lambda x : x.max() - x.min()],
              'POPESTIMATE2015' : [np.mean, lambda x : x.max() - x.min()]}
g.agg(minhas_agg).head(10)

Veja como obtivemos apenas as somas para os nascimentos e apenas a média e a amplitude para a quantidade de populão.

Além disso, perceba como apenas as colunas que eu informei apareceram (as colunas de 2010, 2011 e 2012 foram ignoradas).

Por fim, note que para ter mais de uma função por coluna, novamente usamos uma lista, como fazíamos antes.

Mas novamente tivemos um problema: a função lambda está sem nome. Será que você consegue resolver isso? 

**Exercício:** 
*Pegue tudo o que já aprendemos e crie uma agregação similar a anterior, mas com nomes 'soma', 'media' e 'amplitude'.*

# Transformações

Vamos passar agora para o segundo tópico das nossas aplicações.

Ao contrário de uma agregação, a transformação é uma operação que retorna uma tabela de mesmo tamanho (`shape`) que a tabela original.

Por exemplo, suponha que eu tenho uma tabela e que eu quero dividir todos os valores por dois. Isso é uma transformação, pois a tabela resultante vai ter o mesmo número de linha e colunas da original.

Utilizamos a função `transform` para transformar nossos grupos.

In [ ]:
df2

In [ ]:
df2.groupby(duas_letras).get_group('Ma')

In [ ]:
df2.groupby(duas_letras).transform(np.max)

![image](https://github.com/Giatroo/BeeData_GroupBy-in-Pandas/blob/main/groupby-transform.jpg?raw=true)

In [ ]:
df2.groupby(duas_letras).groups

### Lidar com valores nulos

Uma aplicação legal do transform é preencher valores nulos com agregações de cada grupo.

Por exemplo, suponha que eu tenha uma tabela de produtos de vários tipos: carros, motos, bicicletas e aviões. E que o preço de um determinado carro está faltando (é `NaN`). Uma opção seria tirar a média dos preços e usar esse valor no lugar do preço faltando. Entranto, isso parece um pouco confuso, pois temos veículos bem diferentes (bicicletas que custam muito menos e aviões que custam muito mais) e a média de tudo talvez não seja um valor ideal.

O que nós podemos fazer, então, é tirar a média apenas dos carros (agrupando-os) e usar essa média no lugar o valor faltando. 

Vamos ver essa aplicação na prática:

In [ ]:
precos = np.concatenate((((np.random.randn(6) * 5000) + 40000),
                         ((np.random.randn(4) * 2000) + 30000),
                         ((np.random.randn(3) * 300) + 2000), 
                         ((np.random.randn(7) * 10000) + 1000000))).astype(int)
veiculos_dict = {
    'id' : list(map(lambda x : 'id_' + str(x), range(20))),
    'tipo' : ['Carro']*6 + ['Moto']*4 + ['Bicicleta']*3 + ['Avião']*7,
    'preço' : precos 
}

veiculos = pd.DataFrame(veiculos_dict)
veiculos.loc[0, 'preço'] = np.nan
veiculos.loc[6, 'preço'] = np.nan
veiculos.loc[13, 'preço'] = np.nan
veiculos

Veja como ficaria se preenchesemos com a média de todos os veículos:

In [ ]:
veiculos.fillna(int(veiculos['preço'].mean()))

In [ ]:
def fill_by_group(x):
    # Importante!!! 
    # x é uma coluna!
    # Como a coluna id não suporta a operação 'mean', ela não será retornada
    return x.fillna(int(x.mean()))

# Observe que apenas a coluna preço é retornada, pois não existe média dos ids e 
# a coluna tipo foi utilizada para agrupar 
veiculos.groupby('tipo').transform(fill_by_group)

Agora os preços fazem muito mais sentido.

### Normalização e padronização (normalization e standardization)

Dois processos muito comuns em Machine Learning, normalizar e padronizar os dados significa subtrair a média e dividir pelo desvio padrão.

In [ ]:
veiculos.groupby('tipo').transform(lambda x : (x - x.mean()) / x.std())

Veja que todas essas ferramentas que estamos utilizando são tão poderosas que nos permitem fazer tudo em uma única linha.

**Exercício:**
*Podemos ir além e primeiro preencher os valores nulos com a transformação anterior e depois aplicar a transformação de normalização e padronização. Tente fazer isso =)*

### Filtragem

Para nossa última aplicação, vamos demostrar como podemos usar a função `filter` para manter apenas grupos nos quais estamos interassados. Suponha que temos um banco de dados de vários cursos com as notas de cada aluno daquele curso e que queremos manter apenas os cursos nos quais os alunos tiveram uma média acima de 5.

In [ ]:
ids = np.random.choice(range(10000000, 20000000), size=120)
notas = np.random.randint(11, size=120)
cursos = pd.DataFrame({
    'ids' : ids,
    'curso' : ['Programação I']*30 + ['Cálculo']*30 + ['Probabilidade']*30 + ['Programação II']*30,
    'notas' : notas
})
cursos.sample(10)

A função `filter` recebe o grupo inteiro (diferente de antes, que eram as colunas) e deve retornar `True` ou `False`. Se o grupo retornar `True`, ele será mantido.

In [ ]:
def media_maior(df):
    # df é um DataFrame!
    #print(type(df))
    media = df['notas'].mean()
    nome = df['curso'].iloc[0]
    print(f'A média do curso {nome} é {media}.')
    return media > 5

cursos.groupby('curso').filter(media_maior).sample(10)

### Conclusão e aprofundamento

Por hoje é tudo =)

Espero que você tenha gostado e aprendido bastante. 

Concluímos que a função `groupby` é incrivelmente útil e poderosa, resolvendo diversos problemas diferentes. 

Vimos que existem muitas formas de se realizar um agrupamento com ela. 

Depois, vimos a utilidade da função `agg` e as diferentes formas de se trabalhar com ela.

Por fim, vimos exemplos de outras duas funções utilizadas com agrupamentos: `transform` e `filter`. 

<br>

Caso queira se aprofundar e treinar um pouco o que vimos hoje, seguem alguns materiais adicionais:


[Documentação groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) <br>
[Documentação agg](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) <br>
[Documentação transform](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html) <br>
[Documentação filter](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.filter.html) <br>

[Guia do usuário sobre groupby](https://pandas.pydata.org/docs/user_guide/groupby.html)

Além disso, recomendo fortemente o livro *Python for Data Analysis* do Wes McKinney, criador do `pandas`. Ele foi um guia muito bom para eu aprender **muito** sobre a biblioteca e tem um capítulo só para agrupamentos.

Por fim, tem dois cursos que eu recomendo: <br>
[Kaggle](https://www.kaggle.com/learn/pandas) <br>
[Introdution to Data Science in Python](https://www.coursera.org/learn/python-data-analysis) <br>


**~Lucas Paiolla , 16/03/2021**